### Third Version
* The post and pre bug shouldn't be separated. 

### Second Version

In [32]:
import os, re
import pandas as pd

def read_through_all_the_files_in_a_path_choose_pattern(PATH, pattern_used = False):
    """
    You can choose whether you want to use regular expression to extract the bugID pattern.
    """
    root_PATH = PATH.split("\\")[-1]
    with open("{}_no_pattern.csv".format(root_PATH), "w") as rp:
        rp.write("{},{},{},{}\n".format("file", "fileVersion", "date","bugID"))
        group = os.walk(PATH)
        for path, directory, filelist in group:
            for file in filelist:
                try:
                    ## skip the non-java code:
                    if ".java,v" not in file:
                            continue
                    ## get the lines from original files.
                    lines = None
                    with open(os.path.join(path, file), "r", encoding="utf-8") as f:
                        lines = f.readlines()
                    ## skip the non-java file
                    ## this is the latest version of this file.
                    # print(re.split("\s", lines[0].replace(";", ""))[1])
                    log_start = False
                    version_extraction = True
                    version_start = False
                    ## the dict: versions is used to store the version info:
                    ## versions[a_certain_version] = {
                    ##     "date": None,
                    ##     "bugs": []
                    ## }
                    versions = {}
                    current_version = None
                    version_date = False
                    ##
                    for l in lines:
                        line = l.strip().lower()
                        ## starts from version_extraction, we start to extract version meta info
                        if version_extraction:
                            ## get the version line.
                            if line == "":
                                version_start = True
                            if line != "" and version_start == True:
                                version_start = False
                                current_version = line
                                versions[current_version] = {
                                    "date": None,
                                    "bugs": []
                                }
                                version_date = True
                            ## deal with date line. Some other meta info contained in this line as well. Parse them if needed.
                            if "date" in line and version_date == True:
                                version_date = False
                                date = re.split(r"\s*", line)[1].replace(";", "")
                                versions[current_version]["date"] = date
                            ## end of version meta info zone
                            if line == "desc":
                                version_start = False
                                version_extraction = False
                                current_version = None
                        ## lets' see if there is any version number line in recorded in versions.
                        if line in versions:
                            current_version = line
                        ## add the bugIDs, the bugID will be stored according to version.
                        if log_start:
                            ## no pattern is used
                            if not pattern_used:
                                rst_stage_1 = re.findall("(\d{4,})", line)
                                rst_stage_1 = [int(o) for o in rst_stage_1]
                                versions[current_version]["bugs"].extend(rst_stage_1)
                            ## some pattern is used
                            else:
                                if "fix" in line or "bug" in line: ## the condition here can be added.
                                    ## stage 1: the bug number will be like this: xxxx(space)(a series of numbers)(perhaps here are some other spaces)
                                    rst_stage_1 = re.findall(" (\d{4,})[\s]*", line)
                                    rst_stage_1 = [int(o) for o in rst_stage_1]
                                    versions[current_version]["bugs"].extend(rst_stage_1)
                                    ## stage 2: we want to see if there is any bug id in url.
                                    urls = re.findall("https://bugs.eclipse.org/bugs[\s\S]*", line)
                                    for url in urls:
                                        rst_stage_2 = re.findall("(\d{4,})", url)
                                        rst_stage_2 = [int(o) for o in rst_stage_2]
                                        versions[current_version]["bugs"].extend(rst_stage_2)
                        ## log starts from here.
                        if line == "log":
                            log_start = True
                        ## log ends here.
                        elif line == "@":
                            log_start = False
                    ## get rid of some useless stuff and then print the info.
                    del versions["desc"]
                    for version in versions:
                        version_info = versions[version]
                        date = version_info["date"]
                        version_info["bugs"] = list(set(version_info["bugs"]))
                        for bug in version_info["bugs"]:
                            rp.write(
                                "{},{},{},{}\n".format(
                                    root_PATH + os.path.join(path, file).replace(PATH, "").replace(",v", ""),
                                    version,
                                    date,
                                    bug
                                )
                            )
                except:
                    print("failed: {}".format(os.path.join(path, file)))
    return

def read_through_all_the_files_to_get_release_and_commit_No_1(PATH, pattern_used = False):
    """
    You can choose whether you want to use regular expression to extract the bugID pattern.
    """
    root_PATH = PATH.split("\\")[-1]
    with open("{}_mainRelease_fastRelease_commit.csv".format(root_PATH), "w") as rp:
        rp.write("{},{},{},{}\n".format("file", "releaseNo", "fastReleaseNo", "commitNo"))
        group = os.walk(PATH)
#         file_rel_commit = []
        for path, directory, filelist in group:
            for file in filelist:
                try:
                    ## skip the non-java code:
                    if ".java,v" not in file:
                            continue
                    ## get the lines from original files.
                    lines = None
                    with open(os.path.join(path, file), "r", encoding="utf-8") as f:
                        lines = f.readlines()
                    ##
                    symbols_start = False
                    release_commit = []
                    for l in lines:
                        line = l.strip().lower()
                        if line == "symbols":
                            symbols_start = True
                            continue
                        if not symbols_start:
                            continue
                        if line == "locks; strict;":
                            symbols_start = False
                            continue
                        ### dealing with symbols
                        release_commit.append(l.strip())

                    current_version = None
                    for vstr in reversed(release_commit):
                        if ":" not in vstr:
                            continue
                        vstr = vstr.replace(";", "")
                        releaseNo, commitNo = vstr.split(":")
                        if re.findall("^R[\d_]+$", releaseNo):
                            current_version = releaseNo
                        if current_version != None:
                            rp.write(
                                "{},{},{},{}\n".format(
                                    root_PATH + os.path.join(path, file).replace(PATH, "").replace(",v", ""),
                                    current_version,
                                    releaseNo,
                                    commitNo
                                )
                            )
                except:
                    print("failed: {}".format(os.path.join(path, file)))
            # break
    return

def parse_file_to_map_cvsFileName_packageName(PATH):
    root_PATH = PATH.split("\\")[-1]
    ## java files will be stored into this folder.
    group = os.walk(PATH)
    with open(f"{root_PATH}_map_filePath_packageFileName.csv", "w") as mfp:
        mfp.write("file,pkgFile\n")
        for path, directory, filelist in group:
            for file in filelist:
                try:
                    ###########################3
                    if ".java,v" not in file:
                        continue
                    ## get the lines from original files.
                    with open(os.path.join(path, file), "r", encoding="utf-8",
                              errors="ignore") as f:  # , encoding="utf-8"
                        lines = f.readlines()
                    ## skip the non-java file
                    ## this is the latest version of this file.
                    # print(re.split("\s", lines[0].replace(";", ""))[1])
                    ##
                    for l in lines:
                        line = l.strip()  # .lower()
                        if "package " not in line:
                            continue
                        match_rst = re.match("^package [\s\S]+;$", line)
                        if match_rst == None:
                            continue
                        ###
                        package_name = match_rst.group().split()[1][0:-1]
                        java_file_name = file.replace(",v", "")  # code_file.split("-")[-1]
                        mfp.write(
                            "{},{}.{}\n".format(
                                root_PATH + os.path.join(path, file).replace(PATH, "").replace(",v", ""),
                                # .replace(os.sep, "#"),
                                package_name,
                                java_file_name
                            )
                        )
                        break
                    ##########################
                except:
                    print("failed with utf-8: {}\n".format(os.path.join(path, file)))
    return

def change_versions(x):
    if x[-2:] == ".0":
        return "R{}".format( "_".join(   x[0:-2].split(".")) )
    else:
        return "R{}".format( "_".join(   x.split(".")) )

def merge_tables_1(TAB_PATH_1, TAB_PATH_2, TAB_PATH_3, Bug_List, how = "left"):
    """
    Dates in DataFrame, 
    """
    df1 = pd.read_csv(TAB_PATH_1)
    df1["date"] = df1["date"].map(lambda x: x[0:10].replace(".", "-"))  #pd.to_datetime(, format="%Y.%m.%d")
    df2 = pd.read_csv(TAB_PATH_2)
    df3 = pd.read_csv(TAB_PATH_3)
    df4 = pd.read_csv(Bug_List)
    ####
    df1.rename(columns={"fileVersion": "commitNo"}, inplace= True)
    df3.rename(columns={"Version": "releaseNo"}, inplace=True)
    df3["releaseNo"] = df3["releaseNo"].map(lambda x: change_versions(x))
    df4 = df4[["Bug ID", "Changed"]]
    df4["Changed"] = df4["Changed"].map(lambda x: x[0:10])
    df4.rename(columns={"Bug ID": "bugID"}, inplace=True)
    ####
    _ = pd.merge(df1,df2,on=['file','commitNo'],how= "left")
    _1 = pd.merge(_, df4, on = ["bugID"], how = "left")
    _1 = _1[["file", "releaseNo", "fastReleaseNo", "commitNo", "bugID", "Changed"]]
    df_merge = pd.merge(_1, df3, on = ["releaseNo"], how = how)
    ###
    df_merge["Date"] = pd.to_datetime(df_merge["Date"], format = '%Y-%m-%d')
    df_merge["Changed"] = pd.to_datetime(df_merge["Changed"], format = '%Y-%m-%d')
    ##
    df_merge["Changed-Date"] = df_merge["Changed"] - df_merge["Date"]
    ##
    pre_bug = df_merge[(df_merge['Changed-Date'].dt.days > -180) & (df_merge['Changed-Date'].dt.days <= 0)]
    post_bug = df_merge[(df_merge['Changed-Date'].dt.days > 0) & (df_merge['Changed-Date'].dt.days <= 180)]
    return df_merge, pre_bug, post_bug

## -------------------------------------------------------------------------
## bug extraction code only need to be run once and can be used forever.
## no matter the versions of eclipse
## ---------------------------------------------------------------------------------
# read_through_all_the_files_in_a_path_choose_pattern(r"J:\EECS6444\Raw_Data\cvs\eclipse")
# read_through_all_the_files_to_get_release_and_commit_No_1(r"J:\EECS6444\Raw_Data\cvs\eclipse")
## download the bug id info files from: https://bugs.eclipse.org/bugs/query.cgi?format=advanced
df, pre_bug, post_bug = merge_tables_1(
    r"eclipse_no_pattern.csv",
    r"eclipse_mainRelease_fastRelease_commit.csv",
    r"release-info.csv",
    r"Bugzilla/Whole.csv",
    "inner"
)
pre_bug.to_csv("eclipse_pre_bug.csv", index=False)
post_bug.to_csv("eclipse_post_bug.csv", index=False)
# parse_file_to_map_cvsFileName_packageName(r"J:\EECS6444\Raw_Data\cvs\eclipse")

failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.resources\bundles\org.eclipse.ui.ide\src\org\eclipse\ui\internal\ide\StringMatcher.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.resources\bundles\org.eclipse.ui.ide\src\org\eclipse\ui\internal\views\markers\MarkerCopyHandler.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.resources\bundles\org.eclipse.ui.ide\src\org\eclipse\ui\views\navigator\StringMatcher.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.resources\tests\org.eclipse.core.tests.resources\src\org\eclipse\core\tests\filesystem\SymlinkTest.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.resources\tests\org.eclipse.core.tests.resources\src\org\eclipse\core\tests\resources\content\IContentTypeManagerTest.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.swt\examples\gef.e4\org.eclipse.draw2d\src\org\eclipse\draw2d\PrintFigureOperation.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclip

failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.emf.javascript\src\org\eclipse\e4\emf\ecore\javascript\Script.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.emf.javascript\src\org\eclipse\e4\emf\javascript\views\Attic\AbstractEditingDomainView.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.emf.javascript\src\org\eclipse\e4\emf\javascript\views\Attic\JavascriptView.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\layouts\Layout.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\layouts\LayoutData.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\layouts\LayoutsFactory.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src

failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\AbstractComposite.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\Browser.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\Button.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\CheckBox.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\ComboBox.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\Composite.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\Control.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.

failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.builder\src\org\eclipse\e4\tm\stringconverter\CompositeStringConverter.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.builder\src\org\eclipse\e4\tm\stringconverter\StringConversion.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.builder\src\org\eclipse\e4\tm\stringconverter\StringConverter.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.builder\src\org\eclipse\e4\tm\stringconverter\StringConverterContext.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.builder\src\org\eclipse\e4\tm\stringconverters\AbstractClassStringConverter.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.builder\src\org\eclipse\e4\tm\stringconverters\ArrayStringConverter.java,v
failed: J:\EECS6444\R

failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4-incubator\jdt\org.eclipse.jdt.compiler.as.tests\src\org\eclipse\jdt\compiler\as\tests\Attic\ActionScriptGenerationTest.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4-incubator\jdt\org.eclipse.jdt.compiler.as.tests\src\org\eclipse\jdt\compiler\as\tests\Attic\ActionScriptGenerationTest2.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4-incubator\ui\org.eclipse.e4.demo.e4photo\src\org\eclipse\e4\demo\e4photo\Exif.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\org.eclipse.core.tests.runtime\src\org\eclipse\core\tests\runtime\content\IContentTypeManagerTest.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\org.eclipse.equinox.log\src\org\eclipse\equinox\log\ExtendedLogEntry.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\org.eclipse.equinox.log\src\org\eclipse\equinox\log\ExtendedLogReaderService.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\org.eclipse.equinox.log\src\o

...

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.externaltools\External Tools Base\org\eclipse\ui\externaltools\variable\Attic\ResourceLocExpander.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.externaltools\External Tools Base\org\eclipse\ui\externaltools\variable\Attic\ResourceLocVariableComponent.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.externaltools\External Tools Base\org\eclipse\ui\externaltools\variable\Attic\ResourceLocVariableLocation.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.externaltools\External Tools Base\org\eclipse\ui\externaltools\variable\Attic\ResourceProjectExpander.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.externaltools\External Tools Base\org\eclipse\ui\externaltools\variable\Attic\SpecificFileLocVariableComponent.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.externaltools\External Tools Base\org\eclipse\ui\externaltools\variable\Attic\SpecificFileResourceCompone

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.intro\src\org\eclipse\ui\internal\intro\impl\model\IntroLink.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.intro\src\org\eclipse\ui\internal\intro\impl\model\IntroModelRoot.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.intro\src\org\eclipse\ui\internal\intro\impl\model\IntroSearchParticipant.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.intro\src\org\eclipse\ui\internal\intro\impl\model\IntroTheme.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.intro\src\org\eclipse\ui\internal\intro\impl\model\loader\ModelLoaderUtil.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.intro\src\org\eclipse\ui\internal\intro\impl\model\url\IntroURL.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.intro\src\org\eclipse\ui\internal\intro\impl\presentations\BrowserIntroPartImplementation.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.intro\src\or

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.views\src\org\eclipse\ui\views\properties\PropertySheetEntry.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.views\src\org\eclipse\ui\views\tasklist\Attic\FiltersAction.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.views\src\org\eclipse\ui\views\tasklist\Attic\FiltersDialog.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.views\src\org\eclipse\ui\views\tasklist\Attic\ITaskListHelpContextIds.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.views\src\org\eclipse\ui\views\tasklist\Attic\MarkCompletedAction.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.views\src\org\eclipse\ui\views\tasklist\Attic\SelectAllTasksAction.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.views\src\org\eclipse\ui\views\tasklist\Attic\TaskList.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.views\src\org\eclipse\ui\views\tasklist\Attic\TaskListContentPro

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\dialogs\Attic\InstallCapabilityStep.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\dialogs\Attic\MultiStepCapabilityWizard.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\dialogs\Attic\MultiStepConfigureWizardPage.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\dialogs\Attic\MultiStepReviewWizardPage.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\dialogs\Attic\MultiStepWizard.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\dialogs\Attic\MultiStepWizardDialog.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\dialogs\Attic\NewProjectWi

failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ds\org.eclipse.pde.ds.ui\src\org\eclipse\pde\internal\ds\ui\editor\Attic\DSEditReferenceDialog.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ds\org.eclipse.pde.ds.ui\src\org\eclipse\pde\internal\ds\ui\editor\Attic\DSImplementationDetails.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ds\org.eclipse.pde.ds.ui\src\org\eclipse\pde\internal\ds\ui\editor\Attic\DSMasterTreeSection.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ds\org.eclipse.pde.ds.ui\src\org\eclipse\pde\internal\ds\ui\editor\Attic\DSPage.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ds\org.eclipse.pde.ds.ui\src\org\eclipse\pde\internal\ds\ui\editor\Attic\DSPropertiesDetails.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ds\org.eclipse.pde.ds.ui\src\org\eclipse\pde\internal\ds\ui\editor\Attic\DSPropertyDetails.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ds\org.eclipse.pde.ds.ui\src\org\eclipse\pde\internal\ds\ui\editor\Attic\DSProvideDetails.java

### First version

In [15]:
import os, re
import pandas as pd

def read_through_all_the_files_in_a_path_choose_pattern(PATH, pattern_used = False):
    """
    You can choose whether you want to use regular expression to extract the bugID pattern.
    """
    root_PATH = PATH.split("\\")[-1]
    with open("{}_no_pattern.csv".format(root_PATH), "w") as rp:
        rp.write("{},{},{},{}\n".format("file", "fileVersion", "date","bugID"))
        group = os.walk(PATH)
        for path, directory, filelist in group:
            for file in filelist:
                try:
                    ## skip the non-java code:
                    if ".java,v" not in file:
                            continue
                    ## get the lines from original files.
                    lines = None
                    with open(os.path.join(path, file), "r", encoding="utf-8") as f:
                        lines = f.readlines()
                    ## skip the non-java file
                    ## this is the latest version of this file.
                    # print(re.split("\s", lines[0].replace(";", ""))[1])
                    log_start = False
                    version_extraction = True
                    version_start = False
                    ## the dict: versions is used to store the version info:
                    ## versions[a_certain_version] = {
                    ##     "date": None,
                    ##     "bugs": []
                    ## }
                    versions = {}
                    current_version = None
                    version_date = False
                    ##
                    for l in lines:
                        line = l.strip().lower()
                        ## starts from version_extraction, we start to extract version meta info
                        if version_extraction:
                            ## get the version line.
                            if line == "":
                                version_start = True
                            if line != "" and version_start == True:
                                version_start = False
                                current_version = line
                                versions[current_version] = {
                                    "date": None,
                                    "bugs": []
                                }
                                version_date = True
                            ## deal with date line. Some other meta info contained in this line as well. Parse them if needed.
                            if "date" in line and version_date == True:
                                version_date = False
                                date = re.split(r"\s*", line)[1].replace(";", "")
                                versions[current_version]["date"] = date
                            ## end of version meta info zone
                            if line == "desc":
                                version_start = False
                                version_extraction = False
                                current_version = None
                        ## lets' see if there is any version number line in recorded in versions.
                        if line in versions:
                            current_version = line
                        ## add the bugIDs, the bugID will be stored according to version.
                        if log_start:
                            ## no pattern is used
                            if not pattern_used:
                                rst_stage_1 = re.findall("(\d{4,})", line)
                                rst_stage_1 = [int(o) for o in rst_stage_1]
                                versions[current_version]["bugs"].extend(rst_stage_1)
                            ## some pattern is used
                            else:
                                if "fix" in line or "bug" in line: ## the condition here can be added.
                                    ## stage 1: the bug number will be like this: xxxx(space)(a series of numbers)(perhaps here are some other spaces)
                                    rst_stage_1 = re.findall(" (\d{4,})[\s]*", line)
                                    rst_stage_1 = [int(o) for o in rst_stage_1]
                                    versions[current_version]["bugs"].extend(rst_stage_1)
                                    ## stage 2: we want to see if there is any bug id in url.
                                    urls = re.findall("https://bugs.eclipse.org/bugs[\s\S]*", line)
                                    for url in urls:
                                        rst_stage_2 = re.findall("(\d{4,})", url)
                                        rst_stage_2 = [int(o) for o in rst_stage_2]
                                        versions[current_version]["bugs"].extend(rst_stage_2)
                        ## log starts from here.
                        if line == "log":
                            log_start = True
                        ## log ends here.
                        elif line == "@":
                            log_start = False
                    ## get rid of some useless stuff and then print the info.
                    del versions["desc"]
                    for version in versions:
                        version_info = versions[version]
                        date = version_info["date"]
                        version_info["bugs"] = list(set(version_info["bugs"]))
                        for bug in version_info["bugs"]:
                            rp.write(
                                "{},{},{},{}\n".format(
                                    root_PATH + os.path.join(path, file).replace(PATH, "").replace(",v", ""),
                                    version,
                                    date,
                                    bug
                                )
                            )
                except:
                    print("failed: {}".format(os.path.join(path, file)))
    return

def read_through_all_the_files_to_get_release_and_commit_No(PATH, pattern_used = False):
    """
    You can choose whether you want to use regular expression to extract the bugID pattern.
    """
    root_PATH = PATH.split("\\")[-1]
    with open("{}_release_commit.csv".format(root_PATH), "w") as rp:
        rp.write("{},{},{}\n".format("file", "releaseNo", "commitNo"))
        group = os.walk(PATH)
        for path, directory, filelist in group:
            for file in filelist:
                try:
                    ## skip the non-java code:
                    if ".java,v" not in file:
                            continue
                    ## get the lines from original files.
                    lines = None
                    with open(os.path.join(path, file), "r", encoding="utf-8") as f:
                        lines = f.readlines()
                    ##
                    symbols_start = False
                    for l in lines:
                        line = l.strip().lower()
                        if line == "symbols":
                            symbols_start = True
                            continue
                        if not symbols_start:
                            continue
                        if line == "locks; strict;":
                            symbols_start = False
                            continue
                        ### dealing with symbols
                        releaseNo, commitNo = l.strip().split(":")
                        rp.write(
                            "{},{},{}\n".format(
                                root_PATH + os.path.join(path, file).replace(PATH, "").replace(",v", ""),
                                releaseNo,
                                commitNo
                            )
                        )
                except:
                    print("failed: {}".format(os.path.join(path, file)))
    return

def change_versions(x):
    if x[-2:] == ".0":
        return "R{}".format( "_".join(   x[0:-2].split(".")) )
    else:
        return "R{}".format( "_".join(   x.split(".")) )

def merge_tables(TAB_PATH_1, TAB_PATH_2, TAB_PATH_3, Bug_List, how = "outer"):
    """
    Dates in DataFrame, 
    """
    df1 = pd.read_csv(TAB_PATH_1)
    df1["date"] = df1["date"].map(lambda x: x[0:10].replace(".", "-"))  #pd.to_datetime(, format="%Y.%m.%d")
    df2 = pd.read_csv(TAB_PATH_2)
    df3 = pd.read_csv(TAB_PATH_3)
    df4 = pd.read_csv(Bug_List)
    ####
    df1.rename(columns={"fileVersion": "commitNo"}, inplace= True)
    df3.rename(columns={"Version": "releaseNo"}, inplace=True)
    df3["releaseNo"] = df3["releaseNo"].map(lambda x: change_versions(x))
    df4 = df4[["Bug ID", "Changed"]]
    df4["Changed"] = df4["Changed"].map(lambda x: x[0:10])
    df4.rename(columns={"Bug ID": "bugID"}, inplace=True)
    ####
    _ = pd.merge(df1,df2,on=['file','commitNo'],how= how)
    _1 = pd.merge(_, df3, on = ["releaseNo"], how = how)
    df_merge = pd.merge(_1, df4, on = ["bugID"], how = how)
    df_merge["date"] = pd.to_datetime(df_merge["date"], format = '%Y-%m-%d')
    df_merge["Date"] = pd.to_datetime(df_merge["Date"], format = '%Y-%m-%d')
    df_merge["Changed"] = pd.to_datetime(df_merge["Changed"], format = '%Y-%m-%d')
    ##
    df_merge["Changed-Date"] = df_merge["Changed"] - df_merge["Date"]
    ##
    pre_bug = df_merge[(df_merge['Changed-Date'].dt.days > -180) & (df_merge['Changed-Date'].dt.days <= 0)]
    post_bug = df_merge[(df_merge['Changed-Date'].dt.days > 0) & (df_merge['Changed-Date'].dt.days <= 180)]
    return df_merge, pre_bug, post_bug

def parse_file_to_map_cvsFileName_packageName(PATH):
    root_PATH = PATH.split("\\")[-1]
    ## java files will be stored into this folder.
    group = os.walk(PATH)
    with open(f"{root_PATH}_map_filePath_packageFileName.csv", "w") as mfp:
        mfp.write("file,pkgFile\n")
        for path, directory, filelist in group:
            for file in filelist:
                try:
                    ###########################3
                    if ".java,v" not in file:
                        continue
                    ## get the lines from original files.
                    with open(os.path.join(path, file), "r", encoding="utf-8",
                              errors="ignore") as f:  # , encoding="utf-8"
                        lines = f.readlines()
                    ## skip the non-java file
                    ## this is the latest version of this file.
                    # print(re.split("\s", lines[0].replace(";", ""))[1])
                    ##
                    for l in lines:
                        line = l.strip()  # .lower()
                        if "package " not in line:
                            continue
                        match_rst = re.match("^package [\s\S]+;$", line)
                        if match_rst == None:
                            continue
                        ###
                        package_name = match_rst.group().split()[1][0:-1]
                        java_file_name = file.replace(",v", "")  # code_file.split("-")[-1]
                        mfp.write(
                            "{},{}.{}\n".format(
                                root_PATH + os.path.join(path, file).replace(PATH, "").replace(",v", ""),
                                # .replace(os.sep, "#"),
                                package_name,
                                java_file_name
                            )
                        )
                        break
                    ##########################
                except:
                    print("failed with utf-8: {}\n".format(os.path.join(path, file)))
    return

## -------------------------------------------------------------------------
## bug extraction code only need to be run once and can be used forever.
## no matter the versions of eclipse
## ---------------------------------------------------------------------------------
# read_through_all_the_files_in_a_path_choose_pattern(r"J:\EECS6444\Raw_Data\cvs\eclipse")
read_through_all_the_files_to_get_release_and_commit_No(r"J:\EECS6444\Raw_Data\cvs\eclipse")
## download the bug id info files from: https://bugs.eclipse.org/bugs/query.cgi?format=advanced
# df, pre_bug, post_bug = merge_tables(
#     r"eclipse_no_pattern.csv",
#     r"eclipse_release_commit.csv",
#     r"release-info.csv",
#     r"Bugzilla/Whole.csv",
#     "inner"
# )
# pre_bug.to_csv("eclipse_pre_bug.csv", index=False)
# post_bug.to_csv("eclipse_post_bug.csv", index=False)
# parse_file_to_map_cvsFileName_packageName(r"J:\EECS6444\Raw_Data\cvs\eclipse")

failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.resources\bundles\org.eclipse.ui.ide\src\org\eclipse\ui\internal\ide\StringMatcher.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.resources\bundles\org.eclipse.ui.ide\src\org\eclipse\ui\internal\views\markers\MarkerCopyHandler.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.resources\bundles\org.eclipse.ui.ide\src\org\eclipse\ui\views\navigator\StringMatcher.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.resources\tests\org.eclipse.core.tests.resources\src\org\eclipse\core\tests\filesystem\SymlinkTest.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.resources\tests\org.eclipse.core.tests.resources\src\org\eclipse\core\tests\resources\content\IContentTypeManagerTest.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.swt\examples\gef.e4\org.eclipse.draw2d\src\org\eclipse\draw2d\PrintFigureOperation.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclip

failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\layouts\Layout.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\layouts\LayoutData.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\layouts\LayoutsFactory.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\layouts\LayoutsPackage.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\layouts\impl\LayoutDataImpl.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\layouts\impl\LayoutImpl.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\layouts\impl\LayoutsFactoryImpl.java,v
failed: J:\EECS64

failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\PushButton.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\Shell.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\SingleSelectionList.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\Tab.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\TabFolder.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\Text.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm\src\org\eclipse\e4\tm\widgets\ToggleButton.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.

failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.graphics\src\edu\umd\cs\piccolox\util\PAbstractLineLocator.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.graphics\src\edu\umd\cs\piccolox\util\PLineLocator.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.graphics\src\edu\umd\cs\piccolox\util\PLocatorsLineLocator.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.ui\src\org\eclipse\e4\tm\ui\AbstractModelContext.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.ui\src\org\eclipse\e4\tm\ui\AbstractModelView.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.ui\src\org\eclipse\e4\tm\ui\Activator.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\e4\org.eclipse.e4.tm\bundles\org.eclipse.e4.tm.ui\src\org\eclipse\e4\tm\ui\ActiveEditor

failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\provisioning\ie.wombat.jbdiff\src\ie\wombat\jbdiff\Attic\Activator.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\provisioning\ie.wombat.jbdiff\src\ie\wombat\jbdiff\Attic\JBDiff.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\provisioning\ie.wombat.jbdiff\src\ie\wombat\jbdiff\Attic\JBPatch.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\provisioning\ie.wombat.jbdiff\src\ie\wombat\jbdiff\Attic\Util.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\provisioning\org.eclipse.equinox.p2.artifact.optimizers\src\org\eclipse\equinox\internal\p2\artifact\optimizers\jardelta\Attic\Optimizer.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\provisioning\org.eclipse.equinox.p2.artifact.optimizers\src\org\eclipse\equinox\internal\p2\artifact\optimizers\jbdiff\Attic\ArtifactKeyDeSerializer.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\equinox-incubator\p

...

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.ide\extensions\org\eclipse\ui\dialogs\WizardNewFolderMainPage.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.ide\src\org\eclipse\ui\ide\dialogs\Attic\ResourceFilterEditDialog.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.ide\src\org\eclipse\ui\ide\undo\CreateFolderOperation.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.ide\src\org\eclipse\ui\internal\ide\ResourceFilterDecorator.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.ide\src\org\eclipse\ui\internal\ide\StringMatcher.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.ide\src\org\eclipse\ui\internal\ide\dialogs\ResourceFilterEditDialog.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.ide\src\org\eclipse\ui\internal\ide\dialogs\ResourceFilterGroup.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.ide\src\org\eclipse\ui\internal\ide\dialogs\Attic\UIResourceFilterDescription.

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.tests\Eclipse UI Tests\org\eclipse\ui\tests\menus\MenusTestSuite.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.tests\Eclipse UI Tests\org\eclipse\ui\tests\menus\ShowViewMenuTest.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.tests\Eclipse UI Tests\org\eclipse\ui\tests\zoom\ActivateTest.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.tests\Eclipse UI Tests\org\eclipse\ui\tests\zoom\OpenEditorTest.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.tests\Eclipse UI Tests\org\eclipse\ui\tests\zoom\ZoomTestCase.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.tests\Eclipse UI Tests\org\eclipse\ui\tests\zoom\ZoomTestSuite.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.tests\Eclipse UI Tests\org\eclipse\ui\tests\zoom\Attic\CloseEditorTest.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.tests\Eclipse UI Tests\org\eclipse\ui\tests\zoom\

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\ActionSetActionBars.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\CloseAllSavedAction.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\EditorActionBars.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\LayoutPart.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\LayoutPartSash.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\LayoutTree.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\LayoutTreeNode.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\PageLayout.java,v
failed: J:\EECS6

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\registry\Attic\Capability.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\registry\Attic\CapabilityRegistry.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\registry\Attic\CapabilityRegistryReader.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\internal\registry\Attic\ICategory.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench\Eclipse UI\org\eclipse\ui\part\CellEditorActionHandler.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench.texteditor\src\org\eclipse\ui\internal\texteditor\EditPosition.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.ui.workbench.texteditor\src\org\eclipse\ui\internal\texteditor\TextEditorPlugin.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclips

failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ui\org.eclipse.pde.core\src\org\eclipse\pde\internal\core\util\Attic\StringMatcher.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ui\org.eclipse.pde.runtime\src\org\eclipse\pde\internal\runtime\PDERuntimeMessages.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ui\org.eclipse.pde.runtime\src\org\eclipse\pde\internal\runtime\registry\RegistryBrowser.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ui\org.eclipse.pde.runtime\src\org\eclipse\pde\internal\runtime\registry\RegistryBrowserContentProvider.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ui\org.eclipse.pde.runtime\src\org\eclipse\pde\internal\runtime\registry\RegistryBrowserLabelProvider.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde\ui\org.eclipse.pde.runtime\src\org\eclipse\pde\internal\runtime\registry\Attic\RegistryBrowserListener.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\pde-incubator\modeling\plugins\org.eclipse.pde.ds\src\org\eclipse\pde\ds\scr\impl\S

-------------------------
## Code Recycle Bin

In [5]:
import os, re
import pandas as pd
def change_versions(x):
    if x[-2:] == ".0":
        return "R{}".format( "_".join(   x[0:-2].split(".")) )
    else:
        return "R{}".format( "_".join(   x.split(".")) )

In [3]:
# TAB_PATH_1 = r"eclipse_no_pattern.csv",
# TAB_PATH_2 = r"eclipse_release_commit.csv",
# TAB_PATH_3 = r"release-info.csv",
# Bug_List = r"Bugzilla/Whole.csv",

df1 = pd.read_csv("eclipse_no_pattern.csv")
df1["date"] = df1["date"].map(lambda x: x[0:10].replace(".", "-"))  #pd.to_datetime(, format="%Y.%m.%d")
df2 = pd.read_csv("eclipse_mainRelease_fastRelease_commit.csv")
df3 = pd.read_csv("release-info.csv")
df4 = pd.read_csv("Bugzilla/Whole.csv")
####

In [6]:
df1.rename(columns={"fileVersion": "commitNo"}, inplace= True)
df3.rename(columns={"Version": "releaseNo"}, inplace=True)
df3["releaseNo"] = df3["releaseNo"].map(lambda x: change_versions(x))
df4 = df4[["Bug ID", "Changed"]]
df4["Changed"] = df4["Changed"].map(lambda x: x[0:10])
df4.rename(columns={"Bug ID": "bugID"}, inplace=True)

In [15]:
df1.head()

,file,commitNo,date,bugID
0,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599
1,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599
2,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599
3,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2010-03-02,299385
4,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599


In [16]:
df2.head()

,file,releaseNo,fastReleaseNo,commitNo
0,eclipse\e4\org.eclipse.e4.compatibility\bundle...,R1_0M3,R1_0M3,1.1
1,eclipse\e4\org.eclipse.e4.compatibility\bundle...,R1_0M4,R1_0M4,1.1
2,eclipse\e4\org.eclipse.e4.compatibility\bundle...,R1_0M4,pre_compat20100302,1.1
3,eclipse\e4\org.eclipse.e4.compatibility\bundle...,R1_0M4,I20100406-0910,1.1
4,eclipse\e4\org.eclipse.e4.compatibility\bundle...,R1_0M4,I20100406-2227,1.1


In [17]:
df3.head()

,releaseNo,Date
0,R4_13,2019-09-18
1,R4_12,2019-06-19
2,R4_11,2019-03-20
3,R4_10,2018-12-19
4,R4_9,2018-09-19


In [18]:
df4.head()

,bugID,Changed
0,551903,2019-10-08
1,409628,2019-06-12
2,548572,2019-06-24
3,321251,2019-06-10
4,484969,2019-07-10


In [20]:
_ = pd.merge(df1,df2,on=['file','commitNo'],how= "left")
_.head()

,file,commitNo,date,bugID,releaseNo,fastReleaseNo
0,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599,R1_0M3,R1_0M3
1,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599,R1_0M4,R1_0M4
2,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599,R1_0M4,pre_compat20100302
3,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599,R1_0M4,I20100406-0910
4,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599,R1_0M4,I20100406-2227


how is "left", not "inner" any more.

In [22]:
_1 = pd.merge(_, df4, on = ["bugID"], how = "left")
_1.head()

,file,commitNo,date,bugID,releaseNo,fastReleaseNo,Changed
0,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599,R1_0M3,R1_0M3,2013-04-26
1,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599,R1_0M4,R1_0M4,2013-04-26
2,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599,R1_0M4,pre_compat20100302,2013-04-26
3,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599,R1_0M4,I20100406-0910,2013-04-26
4,eclipse\e4\org.eclipse.e4.compatibility\bundle...,1.1,2009-12-17,296599,R1_0M4,I20100406-2227,2013-04-26


In [25]:
_1 = _1[["file", "releaseNo", "fastReleaseNo", "commitNo", "bugID", "Changed"]]
_1.head()

,file,releaseNo,fastReleaseNo,commitNo,bugID,Changed
0,eclipse\e4\org.eclipse.e4.compatibility\bundle...,R1_0M3,R1_0M3,1.1,296599,2013-04-26
1,eclipse\e4\org.eclipse.e4.compatibility\bundle...,R1_0M4,R1_0M4,1.1,296599,2013-04-26
2,eclipse\e4\org.eclipse.e4.compatibility\bundle...,R1_0M4,pre_compat20100302,1.1,296599,2013-04-26
3,eclipse\e4\org.eclipse.e4.compatibility\bundle...,R1_0M4,I20100406-0910,1.1,296599,2013-04-26
4,eclipse\e4\org.eclipse.e4.compatibility\bundle...,R1_0M4,I20100406-2227,1.1,296599,2013-04-26


In [27]:
df_merge = pd.merge(_1, df3, on = ["releaseNo"], how = "inner")
df_merge.head()

,file,releaseNo,fastReleaseNo,commitNo,bugID,Changed,Date
0,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,v20060717,1.4,143576,2007-06-07,2006-06-29
1,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,v20060808,1.4,143576,2007-06-07,2006-06-29
2,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,R3_2,1.3,131405,2008-12-05,2006-06-29
3,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,R3_2,1.1,113663,2006-04-05,2006-06-29
4,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,v20060717,1.1,113663,2006-04-05,2006-06-29


In [29]:
####
df_merge["Date"] = pd.to_datetime(df_merge["Date"], format = '%Y-%m-%d')
df_merge["Changed"] = pd.to_datetime(df_merge["Changed"], format = '%Y-%m-%d')
df_merge.head()

,file,releaseNo,fastReleaseNo,commitNo,bugID,Changed,Date
0,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,v20060717,1.4,143576,2007-06-07,2006-06-29
1,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,v20060808,1.4,143576,2007-06-07,2006-06-29
2,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,R3_2,1.3,131405,2008-12-05,2006-06-29
3,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,R3_2,1.1,113663,2006-04-05,2006-06-29
4,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,v20060717,1.1,113663,2006-04-05,2006-06-29


In [30]:
##
df_merge["Changed-Date"] = df_merge["Changed"] - df_merge["Date"]
df_merge.head()

,file,releaseNo,fastReleaseNo,commitNo,bugID,Changed,Date,Changed-Date
0,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,v20060717,1.4,143576,2007-06-07,2006-06-29,343 days
1,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,v20060808,1.4,143576,2007-06-07,2006-06-29,343 days
2,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,R3_2,1.3,131405,2008-12-05,2006-06-29,890 days
3,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,R3_2,1.1,113663,2006-04-05,2006-06-29,-85 days
4,eclipse\e4\org.eclipse.e4.resources\bundles\or...,R3_2,v20060717,1.1,113663,2006-04-05,2006-06-29,-85 days


In [ ]:
##
pre_bug = df_merge[(df_merge['Changed-Date'].dt.days > -180) & (df_merge['Changed-Date'].dt.days <= 0)]
post_bug = df_merge[(df_merge['Changed-Date'].dt.days > 0) & (df_merge['Changed-Date'].dt.days <= 180)]